In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.conv_tasnet as tasnet
from malaya_speech.train.model import enhancement
import tensorflow as tf






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [4]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, (None, 1))
partitioned_x = malaya_speech.tf_featurization.pad_and_partition(x, 8000)
model = tasnet.Model(partitioned_x)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [5]:
logits = tf.expand_dims(model.logits[0], -1)
logits

<tf.Tensor 'ExpandDims:0' shape=(?, ?, 1) dtype=float32>

In [6]:
snr = enhancement.loss.snr(logits, partitioned_x)
sdr = enhancement.loss.sdr(logits, partitioned_x)
sdr_v2 = enhancement.loss.sdr_v2(logits[:,:,0], partitioned_x[:,:,0])


Instructions for updating:
keep_dims is deprecated, use keepdims instead


NameError: name 'EPS' is not defined

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [8]:
import numpy as np

In [9]:
y, sr = malaya_speech.load('../speech/record/savewav_2020-09-06_21-48-43_174842.wav', sr = 22050)

In [10]:
len(y) / sr

11.74

In [16]:
%%time

y_, snr_, sdr_, sdr_v2_ = sess.run([logits, snr, sdr, partitioned_x], 
                                   feed_dict = {x: np.expand_dims(y, -1)})
y_.shape

CPU times: user 43.7 s, sys: 1.57 s, total: 45.3 s
Wall time: 8.74 s


(33, 8000, 1)

In [17]:
snr_, sdr_, sdr_v2_

((0.8001826, -22.73266),
 -9.589455e-08,
 array([[[-0.00570545],
         [-0.01051882],
         [-0.01370371],
         ...,
         [-0.00748609],
         [-0.01006283],
         [-0.01110091]],
 
        [[-0.00962946],
         [-0.0105111 ],
         [-0.01246713],
         ...,
         [-0.40132815],
         [-0.41091236],
         [-0.40693444]],
 
        [[-0.3832167 ],
         [-0.34509313],
         [-0.30196384],
         ...,
         [ 0.19410308],
         [ 0.20203955],
         [ 0.21014789]],
 
        ...,
 
        [[-0.06798829],
         [-0.06250738],
         [-0.05445184],
         ...,
         [-0.14074089],
         [-0.13976824],
         [-0.13747981]],
 
        [[-0.1338544 ],
         [-0.1292843 ],
         [-0.12536876],
         ...,
         [-0.00931244],
         [-0.01260449],
         [-0.01534642]],
 
        [[-0.01858082],
         [-0.0219944 ],
         [-0.02279247],
         ...,
         [ 0.        ],
         [ 0.        ],
     

In [13]:
import IPython.display as ipd

ipd.Audio(y_.reshape((-1)), rate = 22050)

In [15]:
saver = tf.train.Saver()
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [16]:
!ls -lh test

total 71440
-rw-r--r--  1 huseinzolkepli  staff    77B Feb  6 11:58 checkpoint
-rw-r--r--  1 huseinzolkepli  staff    33M Feb  6 11:58 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff    15K Feb  6 11:58 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff   1.7M Feb  6 11:58 model.ckpt.meta


In [17]:
!rm -rf test

In [15]:
np.log(10)

2.302585092994046

In [13]:
def norm(x):
    return np.sum(x ** 2, axis = -1, keepdims = True)

In [14]:
pred = y_
true = sdr_v2_

In [22]:
s_target = (
    np.sum(pred * true, axis = -1, keepdims = True)
    * true
    / (norm(true) + 1e-8)
)
upp = norm(s_target)
low = norm(pred - s_target)

In [26]:
np.log1p(upp / (low + 1e-8))

array([[[ 6.966596 ],
        [ 9.412605 ],
        [10.470172 ],
        ...,
        [ 8.052483 ],
        [ 9.234386 ],
        [ 9.627833 ]],

       [[ 9.05876  ],
        [ 9.40973  ],
        [10.091946 ],
        ...,
        [18.863987 ],
        [17.059498 ],
        [17.701752 ]],

       [[17.158228 ],
        [17.277918 ],
        [17.529257 ],
        ...,
        [18.710932 ],
        [11.62291  ],
        [17.564085 ]],

       ...,

       [[16.262579 ],
        [16.36479  ],
        [15.824572 ],
        ...,
        [18.126446 ],
        [16.094828 ],
        [17.625278 ]],

       [[17.413782 ],
        [17.545792 ],
        [17.29259  ],
        ...,
        [ 8.925489 ],
        [10.134087 ],
        [10.9217205]],

       [[11.685656 ],
        [12.356491 ],
        [12.500486 ],
        ...,
        [ 0.       ],
        [ 0.       ],
        [ 0.       ]]], dtype=float32)